In [1]:
# Markdown: points&rewards.ipynb
import mysql.connector
import pandas as pd
import bcrypt
from datetime import datetime, timedelta

DB_CONFIG = {"host":"localhost","user":"root","password":"40418","database":"aeras"}
def get_conn(): return mysql.connector.connect(**DB_CONFIG)
def now_bd(): return datetime.utcnow() + timedelta(hours=6)

# Utility: award points given distance in meters (per PDF)
def award_points_for_drop(pullerid, distance_m, rideid=None):
    conn = get_conn()
    now = now_bd()
    pending = False
    pts = 0
    if distance_m <= 50:
        pts = 10
    elif distance_m <= 100:
        pts = 5
    else:
        pending = True

    if pending:
        # mark ride as pending review in rides table
        if rideid:
            with conn.cursor() as cur:
                cur.execute("UPDATE rides SET status=%s, drop_time=%s WHERE rideid=%s", ("PENDING_REVIEW", now, rideid))
                conn.commit()
        conn.close()
        return {"pending_review": True}

    # Insert into points table
    exp_date = (now.date() + timedelta(days=180))
    with conn.cursor() as cur:
        cur.execute("INSERT INTO points (pullerid, point, date, exp_date, time, created_at) VALUES (%s,%s,%s,%s,%s,%s)",
                    (pullerid, pts, now.date(), exp_date, now.time(), now))
        # update puller totals
        cur.execute("UPDATE pullers SET points = points + %s, ride_count = ride_count + 1 WHERE pullerid=%s", (pts, pullerid))
        # get new total
        cur.execute("SELECT points FROM pullers WHERE pullerid=%s", (pullerid,))
        new_total = cur.fetchone()[0]
        # insert reward row
        cur.execute("INSERT INTO rewards (pullerid, p_points, redeem, t_points, date, created_at) VALUES (%s,%s,%s,%s,%s,%s)",
                    (pullerid, pts, 0, new_total, now.date(), now))
        # update rides.points_awarded if rideid provided
        if rideid:
            cur.execute("UPDATE rides SET points_awarded=%s, status=%s, drop_time=%s, completion_time=%s WHERE rideid=%s",
                        (pts, "COMPLETED", now, now, rideid))
        conn.commit()
    conn.close()
    return {"pending_review": False, "points_awarded": pts, "new_total": new_total}


In [2]:
# Helper: expire points daily (example function to run via cron)
def expire_points_job():
    conn = get_conn()
    today = now_bd().date()
    # select points that expire today or earlier
    df = pd.read_sql("SELECT * FROM points WHERE exp_date <= %s", conn, params=(today,))
    with conn.cursor() as cur:
        for _, row in df.iterrows():
            pullerid = row['pullerid']
            pts = row['point']
            # deduct from puller.points but ensure non-negative
            cur.execute("UPDATE pullers SET points = GREATEST(points - %s, 0) WHERE pullerid=%s", (pts, pullerid))
            # optionally move to rewards/redeem table or mark as expired
            cur.execute("DELETE FROM points WHERE id=%s", (row['id'],))
        conn.commit()
    conn.close()
    return True


In [3]:
# Leaderboard functions (all metrics)
def get_leaderboard(limit=20):
    conn = get_conn()
    # points, completed rides, avg completion time, avg wait time, reliability
    query = """
    SELECT p.pullerid, p.points, p.ride_count,
           p.avg_wait_time_seconds, p.avg_completion_time_seconds,
           -- reliability: (successful completed rides / rides offered) we estimate success as ride_count / (ride_count + rejects)
           COALESCE(p.points,0) AS total_points
    FROM pullers p
    ORDER BY p.points DESC
    LIMIT %s
    """
    df = pd.read_sql(query, conn, params=(limit,))
    conn.close()
    return df

print(get_leaderboard(10))


Empty DataFrame
Columns: [pullerid, points, ride_count, avg_wait_time_seconds, avg_completion_time_seconds, total_points]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_11084\1770303044.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(limit,))


cursor.execute("ALTER TABLE points CHANGE exp_date date1 VARCHAR(20);")
cursor.execute("ALTER TABLE points CHANGE date exp_date VARCHAR(20);")
cursor.execute("ALTER TABLE points CHANGE date1 date VARCHAR(20);")
conn.commit()